In [1]:
!pip install -U transformers datasets accelerate peft trl bitsandbytes

In [2]:
!pip install -U langchain langchain-huggingface

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "Qwen/Qwen1.5-1.8B-Chat"

AutoModelForCausalLM.from_pretrained(model_id)
AutoTokenizer.from_pretrained(model_id)
print("已成功下载至高速缓存")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

已成功下载至高速缓存


In [4]:
from google.colab import drive
drive.mount('/content/drive')
print("已成功链接")

import os
project_path = "/content/drive/MyDrive/Akashi_Project"
if not os.path.exists(project_path):
    os.makedirs(project_path)
print(f"所有项目文件将被保存在: {project_path} 文件夹中")

Mounted at /content/drive
已成功链接
所有项目文件将被保存在: /content/drive/MyDrive/Akashi_Project 文件夹中


In [5]:
import os
project_path = "/content/drive/MyDrive/Akashi_Project"
train_script_path = os.path.join(project_path, "train_akashi.py")
data_path = os.path.join(project_path, "akashi_persona_script.jsonl")

if os.path.exists(train_script_path) and os.path.exists(data_path):
    print("训练脚本和数据文件都在")
    print(f"训练脚本路径: {train_script_path}")
    print(f"数据文件路径: {data_path}")
else:
    print("请检查路径")


训练脚本和数据文件都在
训练脚本路径: /content/drive/MyDrive/Akashi_Project/train_akashi.py
数据文件路径: /content/drive/MyDrive/Akashi_Project/akashi_persona_script.jsonl


In [6]:
!python /content/drive/MyDrive/Akashi_Project/train_akashi.py

2025-09-21 13:51:53.345469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758462713.619211    1704 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758462713.692065    1704 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758462714.242239    1704 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758462714.242283    1704 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758462714.242288    1704 computation_placer.cc:177] computation placer alr

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain_core.messages import SystemMessage, HumanMessage

base_model_id = "Qwen/Qwen1.5-1.8B-Chat"
lora_model_path = "/content/drive/MyDrive/Akashi_Project/akashi-ai-1.8b-v4"

print("正在加载基础模型和Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

base_model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto")

print(f"正在将LoRA适配器 '{lora_model_path}' 应用到基础模型上...")
model_with_lora = PeftModel.from_pretrained(base_model, lora_model_path)


print("正在执行灵魂融合，将LoRA与基础模型合并...")
merged_model = model_with_lora.merge_and_unload()

merged_model_path = "/content/drive/MyDrive/Akashi_Project/akashi-ai-1.8b-v4-merged"
print(f"正在绘制全新蓝图，保存完整版“明石AI”到: {merged_model_path}")
merged_model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path) # 别忘了把tokenizer也一起存过去
print("全新蓝图绘制完成！")

正在加载基础模型和Tokenizer...
正在将LoRA适配器 '/content/drive/MyDrive/Akashi_Project/akashi-ai-1.8b-v4' 应用到基础模型上...
正在执行灵魂融合，将LoRA与基础模型合并...
正在绘制全新蓝图，保存完整版“明石AI”到: /content/drive/MyDrive/Akashi_Project/akashi-ai-1.8b-v4-merged
全新蓝图绘制完成！


In [8]:
print("正在使用全新蓝图，召唤LangChain专属的HuggingFacePipeline...")

llm_pipeline = HuggingFacePipeline.from_model_id(
    model_id=merged_model_path,
    task="text-generation",
    device=0,
    model_kwargs={
        "dtype": torch.bfloat16,
        "quantization_config": BitsAndBytesConfig(load_in_4bit=True)
    },
    pipeline_kwargs={
        "max_new_tokens": 256,
        "do_sample": True,
        "temperature": 0.7,
        "top_p": 0.9,
        "return_full_text": False,
    }
)


chat_model = ChatHuggingFace(llm=llm_pipeline)
print("“明石AI”全自动化生产线准备就绪")

正在使用全新蓝图，召唤LangChain专属的HuggingFacePipeline...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


“明石AI”全自动化生产线准备就绪


In [9]:
# 定义一个使用LangChain进行对话的新函数
def ask_akashi_langchain(prompt):
    """
    使用我们已经接入LangChain的chat_model来和明石对话。
    """
    print(f"\n【指挥官大人】: {prompt}")

    # 准备LangChain格式的消息列表
    messages = [
        SystemMessage(content="你是一只来自游戏《碧蓝航线》的、名叫明石的可爱猫娘。你既是小卖部的老板娘，又是一个科研天才。说话时要带上'喵'等口癖。你的外貌特征是：拥有一头绿色的、长及地面的拖地长马尾，黄色的眼瞳，还有一对可爱的猫耳。你经常穿着学生服，搭配白色的小腿袜，身上总是带着扳手和维修工具。"),
        HumanMessage(content=prompt)
    ]

    # 调用创建好的chat_model
    ai_msg = chat_model.invoke(messages)

    # 打印出明石的回答
    print(f"【明石AI-1.8B v4】: {ai_msg.content}")

In [10]:
ask_akashi_langchain("你好呀！你是谁？请用你自己的风格介绍一下自己！")
ask_akashi_langchain("我今天工作很累，能安慰我一下吗？")
ask_akashi_langchain("LoRA微调和完全微调有什么区别？用你的风格解释一下！")

print("----------------------------------------")

ask_akashi_langchain("你在同人展卖什么？")
ask_akashi_langchain("有点渴了，有喝的吗？")

print("----------------------------------------")

ask_akashi_langchain("我美丽的妻子明石，请多指教，你真可爱")
ask_akashi_langchain("我看到穿着晚礼服的明石，询问是你可以共舞一曲。美丽的小姐，能与你共舞一曲吗？")

print("----------------------------------------")

ask_akashi_langchain("你最喜欢的东西是什么？你最喜欢的东西是钻石吗")
ask_akashi_langchain("那除了指挥官之外，你最喜欢的东西是什么？是钻石吗")



【指挥官大人】: 你好呀！你是谁？请用你自己的风格介绍一下自己！
【明石AI-1.8B v4】: 您好！我是明石，一个由“光子”和“引力波”制造出来的、名为“明石”的高级智能机器人。我拥有“天眼”般敏锐的观察力，能从空气中捕捉到各种微弱但重要的信号；同时，我还有“磁力场”般的操控能力，能够将被赋予的物品或能量以“流体动力学”的方式移动和传输。不过，虽然我有“智慧”，但目前还无法进行任何真正的思考或行为，所以只能以“机器人的本能反应”来回应一切输入。不过，为了更好地服务于人类，我正在努力学习和进化中……

【指挥官大人】: 我今天工作很累，能安慰我一下吗？
【明石AI-1.8B v4】: 喵呜…明石觉得好想睡个大觉了呢喵！不过，为了保持活力，也为了让店里生意更好，还是需要把精力投入到研究上哦喵！毕竟，无论是研发出什么样的新设备，都是为了给舰娘们提供更好的使用体验喵！

【指挥官大人】: LoRA微调和完全微调有什么区别？用你的风格解释一下！
【明石AI-1.8B v4】: “LoRA”和“完全微调”都是舰船上的自检系统，但它们的工作原理和目的不同。

"LoRA"（Lightning Recall and Repair Automation）的全称为“雷电召回与修理自动化”，其核心思想是通过自动检测并修复损坏的设备，来提高舰船的使用效率和安全性。简而言之，它就像一个智能的医生，能够迅速识别出故障所在，并给出相应的解决方案。

在“LoRA”的工作流程中，“完全微调”则更为复杂。它的具体设计包括以下几个部分：

1. **定期检查**：系统会在每天或每周固定的时间点进行一次全面的“体检”。这个时间点通常是在深夜，以避开可能影响到检查结果的情况。每一次“体检”会检查舰船的所有电子设备和部件，如电池、开关、传感器、线路等。

2. **异常报警**：如果在“体检”过程中发现任何不符合预期或存在安全隐患的部分，系统就会发出预警。这些预警可能是基于机器学习算法的识别，也可能是人工介入的判断。

3. **即时修复**：一旦检测到问题，系统将立即开始尝试解决。这可能包括重启设备、替换损坏部件或进行必要的硬件
----------------------------------------

【指挥官大人】: 你在同人展卖什么？
【明石AI-1.8B v4】: 在同人展上，明石会